In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [ ]:
df.shape

(3066766, 19)

Q1: Number of columns: 19 columns in the data

In [ ]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [ ]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
df['duration'].std()

42.594351241920904

Q2: Standard Deviation of the duration: 42.59

In [ ]:
original_len = len(df)
original_len

3066766

In [ ]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
len(df)

3009173

In [ ]:
(len(df) / original_len) * 100

98.1220282212598

Q3: Fraction of records left: ~98%

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

In [ ]:
train_dicts = df[categorical].to_dict(orient='records')

In [ ]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [ ]:
X_train.shape

(3009173, 515)

Q4: Dimensionality of X_train: 515 columns

In [ ]:
target = 'duration'
y_train = df[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

7.649261959890821

Q5: RMSE -> 7.64

In [ ]:
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

In [ ]:
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [ ]:
df_val[categorical] = df_val[categorical].astype(str)

In [ ]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_val = df_val[target].values

In [ ]:
y_pred = lr.predict(X_val)

In [ ]:
mean_squared_error(y_val, y_pred, squared=False)

7.8118156976566695

Q6: RMSE on validation data: 7.81